# Fine-Tuning BERT & GPT (HuggingFace)

In [ ]:
!nvidia-smi
import torch
print(torch.cuda.is_available())

Thu Nov 27 11:22:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             32W /   70W |    3746MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline,
    AutoModelForCausalLM
)
import evaluate

### Fine-Tune BERT (Text Classification)

In [ ]:
# We fine-tune BERT on the SST2 sentiment dataset.
dataset = load_dataset("sst2")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [ ]:
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(batch):
    return tokenizer(batch["sentence"], truncation=True)

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

In [ ]:
args = TrainingArguments(
    output_dir="bert-finetuned-sst2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-600156926.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
lora_trainer = Trainer(
    model=lora_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,

)

/tmp/ipython-input-949479140.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(


In [ ]:
lora_trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.172000,0.248243,0.928899
2,0.119500,0.303642,0.927752


TrainOutput(global_step=8420, training_loss=0.16815285988488277, metrics={'train_runtime': 1055.0719, 'train_samples_per_second': 127.667, 'train_steps_per_second': 7.98, 'total_flos': 2436910441971660.0, 'train_loss': 0.16815285988488277, 'epoch': 2.0})

In [ ]:
trainer.save_model()

##### Evaluate & Test BERT

In [ ]:
clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
clf("This movie was surprisingly good!")
# label_0 = negative , label_1 = positive

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9995071887969971}]

##### Fine-Tune GPT2 (Text Generation)

In [ ]:
gpt_name = "gpt2"
gpt_tok = AutoTokenizer.from_pretrained(gpt_name)
gpt_tok.pad_token = gpt_tok.eos_token

gpt_model = AutoModelForCausalLM.from_pretrained(gpt_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
text_ds = load_dataset("wikitext", "wikitext-2-raw-v1")

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
def tokenize_gpt(batch):
    tokenized_batch = gpt_tok(batch["text"], truncation=True, padding="max_length", max_length=128)
    tokenized_batch["labels"] = tokenized_batch["input_ids"].copy()
    return tokenized_batch

tokenized_text = text_ds.map(tokenize_gpt, batched=True, remove_columns=["text"])
tokenized_text

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3760
    })
})

In [ ]:
gpt_args = TrainingArguments(
    output_dir="gpt2-finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
)

In [ ]:
gpt_trainer = Trainer(
    model=gpt_model,
    args=gpt_args,
    train_dataset=tokenized_text["train"],
    eval_dataset=tokenized_text["validation"],
    tokenizer=gpt_tok,
)

/tmp/ipython-input-742901914.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  gpt_trainer = Trainer(


In [ ]:
gpt_trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.259200,1.265674


TrainOutput(global_step=4590, training_loss=1.281908373531433, metrics={'train_runtime': 1599.1026, 'train_samples_per_second': 22.962, 'train_steps_per_second': 2.87, 'total_flos': 2398530207744000.0, 'train_loss': 1.281908373531433, 'epoch': 1.0})

In [ ]:
gpt_trainer.save_model()

##### Test GPT2 Fine-Tuned

In [ ]:
# GPT2 Generation Test
gen_pipe = pipeline("text-generation", model=gpt_model, tokenizer=gpt_tok, max_length=60)
gen_pipe("Deep learning is a revolutionary field because")[0]["generated_text"]

Device set to use cuda:0


'Deep learning is a revolutionary field because it allows researchers to model and implement new ideas, and to apply them to a larger set of problems in a more realistic way. This allows researchers to study other problems in a much better way, as well as improve'

In [ ]:
#!pip install peft bitsandbytes accelerate -q

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "value"]
)

In [ ]:
lora_model = get_peft_model(model, peft_config)

In [ ]:
print(lora_model.print_trainable_parameters)

<bound method PeftModel.print_trainable_parameters of PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                 

In [ ]:
lora_model.print_trainable_parameters()

trainable params: 591,362 || all params: 110,075,140 || trainable%: 0.5372


In [ ]:
lora_model.train()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the CSV using pandas, skipping bad lines
# The 'engine="python"' argument is sometimes needed for 'on_bad_lines' to work correctly
# Note: 'on_bad_lines' is the recommended argument in newer pandas versions for handling errors.
# If you are using an older pandas version, you might need 'error_bad_lines=False'
try:
    df = pd.read_csv("IMDB Dataset.csv", on_bad_lines='skip', engine='python')
    data_set = Dataset.from_pandas(df)
    print(data_set)
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")
    print("Please check the 'IMDB Dataset.csv' file for corruption or malformed entries.")

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


In [ ]:
data_set = data_set.train_test_split(test_size=0.2)
data_set

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 10000
    })
})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    tokenized_batch = tokenizer(batch["review"], truncation=True, max_length=256, padding=False)
    # Map 'positive' to 1 and 'negative' to 0
    tokenized_batch["labels"] = [1 if s == "positive" else 0 for s in batch["sentiment"]]
    return tokenized_batch

tokenized = data_set.map(tokenize, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
args = TrainingArguments(
    output_dir="IMDB",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    processing_class=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.207000,0.197077,0.927100
2,0.126900,0.256348,0.931300


TrainOutput(global_step=5000, training_loss=0.1702064598083496, metrics={'train_runtime': 3737.6095, 'train_samples_per_second': 21.404, 'train_steps_per_second': 1.338, 'total_flos': 1.05244422144e+16, 'train_loss': 0.1702064598083496, 'epoch': 2.0})

#### When to Fine-Tune vs. Use Embeddings or PEFT